In [2]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score  # Thư viện để tính độ chính xác

# Hàm lấy các mẫu dữ liệu ngẫu nhiên trong đó các phần tử có thể lặp lại (trùng nhau)
def bootstrap(X, y):  # X là DataFrame, y là Series
    n_sample = X.shape[0]
    _id = np.random.choice(n_sample, n_sample, replace=True)  # Chọn ngẫu nhiên
    return X.iloc[_id], y.iloc[_id]  # Lấy mẫu tương ứng từ X và y

# Lớp RandomForest
class RandomForest:
    def __init__(self, n_trees=5, max_depth=10, min_samples_split=2, n_features=None):
        self.n_trees = n_trees  # Số lượng cây cho dự đoán
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.n_features = n_features
        self.trees = []

    def fit(self, X, y):  # X là DataFrame, y là Series
        self.trees = []  # Tạo danh sách chứa các cây
        for i in range(self.n_trees):
            # Tạo một cây quyết định cho mỗi giá trị i
            tree = DecisionTreeClassifier(min_samples_split=self.min_samples_split, max_depth=self.max_depth)
            X_sample, y_sample = bootstrap(X, y)  # Tạo mẫu ngẫu nhiên X và y
            tree.fit(X_sample, y_sample)  # Huấn luyện cây
            self.trees.append(tree)  # Thêm cây vào danh sách

    def predict(self, X):  # X là DataFrame
        # Lấy dự đoán từ từng cây
        arr_pred = np.array([tree.predict(X) for tree in self.trees])
        final_pred = []
        for i in range(arr_pred.shape[1]): 
            sample_pred = arr_pred[:, i]  # Trả loại mảng
            final_pred.append(self.most_value(pd.Series(sample_pred)))  # Dùng hàm most_value
        return np.array(final_pred)  # Trả về giá trị dự đoán sau khi vote n cây

    @staticmethod
    def most_value(series):
        return series.mode()[0]  # Trả về giá trị thường gặp nhất

# Tải dữ liệu Iris
data = load_iris()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)

# Chia tập dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Khởi tạo và huấn luyện mô hình RandomForest
rf = RandomForest(n_trees=10, max_depth=5)
rf.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
predictions = rf.predict(X_test)

# Đánh giá mô hình
accuracy = accuracy_score(y_test, predictions)  # Tính độ chính xác
print(f'Dự đoán: {predictions}')
print(f'Dộ chính xác: {accuracy * 100:.2f}%')


Dự đoán: [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
Dộ chính xác: 100.00%
